In [2]:
import pysam
from clipper.src import readsToWiggle, bam_helpers, call_peak
import numpy
import HTSeq
import pybedtools
import cProfile
import pstats
import cPickle as pickle
import pandas as pd
import StringIO
from IPython.core.display import HTML

In [3]:
%load_ext autoreload
%autoreload 2

In [11]:
#fn = "/projects/ps-yeolab3/encode/analysis/encode_master/204_01_RBFOX2.merged.r2.bam"
#fn = "/projects/ps-yeolab3/encode/analysis/encode_master/240_01_TRA2A.merged.r2.bam"
fn = "/home/gpratt/scratch/ad-hoc/sort_dedup_sort_eCLIPBAF_AAligned.out.XS.r2.bam"
bam_file = pysam.Samfile(fn)

malat1
chr11   AS_STRUCTURE    gene    65265233        65273940        .       +       .       gene_id=ENSG00000251562.3;mrna_length=8708;premrna_length=8708



In [12]:
chrom = "chr9"
start = 66082224
end = 66105835
strand = "-"
reverse_strand = False

In [13]:
subset_reads = list(bam_file.fetch(reference=str(chrom), start=start, end=end))

In [14]:
interesting_reads = """NB501415:61:HLFTYBGXY:1:22103:11722:16791:TGGCGGAAAC""".split("\n")


In [15]:
len(interesting_reads)

1

In [16]:
bam_file = pysam.Samfile(fn)
for read_new in bam_file:
    if read_new.qname in interesting_reads:
        print read_new.qname, read_new.positions[0], read_new.positions[-1], read_new.cigarstring, read_new.is_reverse

NB501415:61:HLFTYBGXY:1:22103:11722:16791:TGGCGGAAAC 66104834 66104847 48S14M True


In [20]:
list(readsToWiggle.get_full_length_cigar(read_new))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [18]:
read_new.positions

[90843930,
 90843931,
 90843932,
 90843933,
 90843934,
 90843935,
 90843936,
 90843937,
 90843938]

In [ ]:
foo = pd.read_table("/home/gpratt/clipper_peaks/new_reads_sorted.txt", names=["read_name"])

In [ ]:
bar = foo.read_name.values[:10]

In [49]:
bam_file = pysam.Samfile(fn)
for read_new in bam_file:
    if read_new.qname in bar:
        print read_new.qname, read_new.positions[0], read_new.positions[-1], read_new.cigarstring

AAAAA:SN1001:449:HGTN3ADXX:1:1106:15296:25665 1996 2036 41M
AAAAA:SN1001:449:HGTN3ADXX:1:1102:16246:70484 2244 2279 36M
AAAAA:SN1001:449:HGTN3ADXX:1:1108:5490:37653 2250 2288 39M
AAAAA:SN1001:449:HGTN3ADXX:1:1109:5522:63644 2265 2307 43M
AAAAA:SN1001:449:HGTN3ADXX:1:1103:20142:9444 2283 2326 44M
AAAAA:SN1001:449:HGTN3ADXX:1:1106:14325:27276 2423 2457 35M
AAAAA:SN1001:449:HGTN3ADXX:1:1104:9550:9448 2767 2808 42M
AAAAA:SN1001:449:HGTN3ADXX:1:1102:3232:43379 2825 2867 43M
AAAAA:SN1001:449:HGTN3ADXX:1:1109:17138:66683 3117 3158 42M
AAAAA:SN1001:449:HGTN3ADXX:1:1108:20092:94210 3161 3204 44M


In [35]:
read_new.is_reverse

False

In [ ]:
129939940 129936835 129979329 129944013 True False
filtered
129939941 129936835 129979327 129944013 True False
filtered
129939961 129936835 129979342 129944013 True False
filtered
129939972 129936835 129979361 129944013 True False
filtered
129939973 129936835 129979361 129944013 True False
filtered
ACCCT:SN1001:449:HGTN3ADXX:1:2108:3979:63673
ACTGC:SN1001:449:HGTN3ADXX:1:2206:16393:49167
CGCGG:SN1001:449:HGTN3ADXX:1:1210:17493:62330
GAAAA:SN1001:449:HGTN3ADXX:2:1108:12376:32706
AAGCG:SN1001:449:HGTN3ADXX:2:2216:2414:69318


In [22]:
read_new.is_reverse

False

In [96]:
read_old.iv.start_d?

In [94]:
bam_fileobj = bam_helpers.Robust_BAM_Reader(fn)
for read_old in bam_fileobj:
    if read_old.read.name in interesting_reads:
        print read_old.read.name, read_old.iv.start_d, read_old.iv.end_d        


GTCGG:SN1001:449:HGTN3ADXX:1:1104:19736:19282 153200465 153200422
CTTCC:SN1001:449:HGTN3ADXX:1:2101:3249:33151 153200472 153200427
TGCTG:SN1001:449:HGTN3ADXX:1:1201:9323:27950 153200471 153200427


In [66]:
def get_full_length_cigar(read):
    for t in read.cigartuples:
        value, times = t
        if value == 3 or value == 2 or value == 1:
            continue
        for x in xrange(times):
            yield value

In [67]:
read_new

In [70]:
read_new.cigarstring

'43M'

In [68]:
len(list(get_full_length_cigar(read_new)))

43

In [71]:
len(read_new.positions)

43

In [69]:
read_new.positions[-1] - read_new.positions[0]

42

In [53]:
explicit_locations = numpy.zeros(shape = end - start, dtype = 'O')

for x in xrange(len(explicit_locations)):
    explicit_locations[x] = set([])

In [54]:
if reverse_strand:
    if strand == "+":
        strand = "-"
    elif strand == "-":
        strand = "+"
(wiggle, jxns, pos_counts,
 lengths, allreads, read_locations) = readsToWiggle.readsToWiggle_pysam(subset_reads, start,
                                          end, strand, "start", False)

In [8]:
wiggle

NameError: name 'wiggle' is not defined

In [9]:
(wiggle, jxns, pos_counts,
 lengths, allreads, read_locations) = readsToWiggle.readsToWiggle_pysam(subset_reads, start,
                                          end, strand, "start", False)

In [10]:
wiggle

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0,
 0,
 0,
 0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,


In [56]:
#This is the worst of hacks, need to factor out pysam eventually
bam_fileobj = bam_helpers.Robust_BAM_Reader(fn)
subset_reads = list(bam_fileobj.fetch(reference=chrom, start=start, end=end))
array_of_reads = call_peak.read_array(subset_reads, start, end)

In [57]:
%%timeit
array_of_reads = call_peak.read_array(subset_reads, start, end)

1 loop, best of 3: 4.41 s per loop


In [58]:
interval = pybedtools.create_interval_from_list("chr11   65266517        65266589        ENSG00000251562.3_0_354 7.70358859683e-295      +       65266553        65266557".split())
bedtool = pybedtools.BedTool([interval])
gi = list(call_peak.bed_to_genomic_interval(bedtool))[0]

In [59]:
htseq_reads = call_peak.get_reads_in_interval(gi, array_of_reads)

In [61]:
foo = htseq_reads.pop()

In [66]:
foo.read.name

'GCCCC:SN1001:449:HGTN3ADXX:1:2112:8706:91070'

In [12]:
bar = sorted(call_peak.read_lengths_from_htseq(htseq_reads))

In [13]:
peak_start = 65266517
peak_stop = 65266589

array_loc_start = peak_start - start
array_loc_stop = peak_stop - start

all_reads = read_locations[array_loc_start:array_loc_stop]    
unique_reads = set().union(*[read for read in all_reads])

In [14]:
def get_aligned_read_length(read):
    #0 is the aligned code in pysam
    return sum([code_len for code, code_len in read.cigartuples if code == 0])
def read_lengths_from_pysam(reads):
    return [get_aligned_read_length(read) for read in reads]

In [15]:
foo = sorted(read_lengths_from_pysam(unique_reads))

In [16]:
foo == bar

True

In [6]:
new_sections = pickle.load(open("/home/gpratt/204_01_RBFOX2.merged.r2.bam_new_small.all_peaks.pickle"))
old_sections = pickle.load(open("/home/gpratt/204_01_RBFOX2.merged.r2.bam_old_small.all_peaks.pickle"))

In [7]:
new_sections

[{'Nclusters': 13,
  'clusters': [chrX	153178775	153178852	ENSG00000089820.11_0_7	-	153178804	153178808,
   chrX	153193668	153193723	ENSG00000089820.11_1_5	-	153193696	153193700,
   chrX	153195517	153195562	ENSG00000089820.11_2_11	-	153195536	153195540,
   chrX	153195961	153196030	ENSG00000089820.11_3_22	-	153195988	153195992,
   chrX	153196233	153196263	ENSG00000089820.11_4_11	-	153196246	153196250,
   chrX	153196460	153196502	ENSG00000089820.11_5_19	-	153196475	153196479,
   chrX	153196853	153196908	ENSG00000089820.11_6_20	-	153196876	153196880,
   chrX	153196943	153196955	ENSG00000089820.11_7_8	-	153196946	153196950,
   chrX	153197606	153197690	ENSG00000089820.11_8_46	-	153197647	153197651,
   chrX	153197691	153197775	ENSG00000089820.11_9_42	-	153197740	153197744,
   chrX	153199659	153199688	ENSG00000089820.11_10_3	-	153199671	153199675,
   chrX	153199876	153199907	ENSG00000089820.11_11_4	-	153199890	153199894,
   chrX	153200234	153200274	ENSG00000089820.11_12_6	-	153200249	15320025

In [52]:
foo = pd.DataFrame(new_sections[0]['sections']).fillna(0)
bar = pd.DataFrame(old_sections[0]['sections']).fillna(0)
foo[~(foo == bar)]

,7914,8406,8512,8607,8752,9364,9402,9536,9661,9829,9947,10032,10317,10473,10581,10680,11114,12708
,7958,8460,8569,8735,8997,9381,9517,9621,9709,9920,10014,10296,10402,10518,10653,10757,11157,12750
expanded_Nreads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
final_smoothing_factor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nPeaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nreads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
smoothing_factor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
threshold,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
bar

,7914,8406,8512,8607,8752,9364,9402,9536,9661,9829,9947,10032,10317,10473,10581,10680,11114,12708
,7958,8460,8569,8735,8997,9381,9517,9621,9709,9920,10014,10296,10402,10518,10653,10757,11157,12750
expanded_Nreads,0,45,44,44,50,32,36,35,0,60,56,56,52,47,41,35,0,0
final_smoothing_factor,0,77.6635,280,107.222,112.091,100,120,120,0,0,0,50.3251,109.806,80,76.6293,24.7831,0,0
nPeaks,0,1,1,1,1,1,0,1,0,0,0,2,1,1,0,0,0,0
nreads,1,8,14,10,17,5,6,6,2,5,5,24,11,4,5,4,1,1
smoothing_factor,0,11,11,11,11,11,11,11,0,0,0,11,11,11,11,11,0,0
threshold,0,4,4,4,4,3,3,3,0,5,5,4,4,4,4,3,0,0
tried,False,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,False,False


In [53]:
foo

,7914,8406,8512,8607,8752,9364,9402,9536,9661,9829,9947,10032,10317,10473,10581,10680,11114,12708
,7958,8460,8569,8735,8997,9381,9517,9621,9709,9920,10014,10296,10402,10518,10653,10757,11157,12750
expanded_Nreads,0,45,44,44,50,32,36,35,0,60,56,56,52,47,41,35,0,0
final_smoothing_factor,0,77.6635,280,107.222,112.091,100,120,120,0,0,0,50.3251,109.806,80,76.6293,24.7831,0,0
nPeaks,0,1,1,1,1,1,0,1,0,0,0,2,1,1,0,0,0,0
nreads,1,8,14,10,17,5,6,6,2,5,5,24,11,4,5,4,1,1
smoothing_factor,0,11,11,11,11,11,11,11,0,0,0,11,11,11,11,11,0,0
threshold,0,4,4,4,4,2,3,3,0,5,5,4,4,4,4,3,0,0
tried,False,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,False,False


In [11]:
old = pd.read_csv("/home/gpratt/peak_results_old.csv", index_col=0)
new = pd.read_csv("/home/gpratt/peak_results_new.csv", index_col=0)

In [13]:
diffs = pd.concat({"new": new[(old != new).apply(any, axis=1)], "old": old[(old != new).apply(any, axis=1)]})
diffs = diffs.swaplevel(0,1).sort_index()

In [17]:
HTML(diffs.T.to_html())

In [58]:
p = pstats.Stats("/home/gpratt/clipper/profile.prof")
p.strip_dirs().sort_stats('cumtime', 'time').print_stats()

In [63]:
p = pstats.Stats("/home/gpratt/profile_rbfox2.prof")
p.strip_dirs().sort_stats('cumtime', 'time').print_stats()

Mon Mar 20 14:33:23 2017    /home/gpratt/profile_rbfox2.prof

         79649626 function calls (79639471 primitive calls) in 239.688 seconds

   Ordered by: cumulative time, internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.098    0.049  234.478  234.478 peakfinder.py:2(<module>)
        1    0.000    0.000  228.895  228.895 peakfinder.py:629(call_main)
        1    0.082    0.082  228.891  228.891 peakfinder.py:535(main)
        1    0.010    0.010  223.463  223.463 peakfinder.py:144(func_star)
        1    0.010    0.010  223.446  223.446 call_peak.py:905(call_peaks)
        3    0.005    0.002  198.766   66.255 call_peak.py:551(peaks)
      411    0.003    0.000  198.576    0.483 call_peak.py:298(fit_univariate_spline)
      411   63.250    0.154  198.573    0.483 fitpack2.py:167(__init__)
      142  135.289    0.953  135.319    0.953 fitpack2.py:233(_reset_nest)
      258    0.001    0.000   97.890    0.379 call_peak.py:338(fit_lo

<pstats.Stats instance at 0x2ad9946cb290>

# run clipper on these

In [2]:
foo = pd.read_table("/home/gpratt/failed_genes.txt", names=['chrom', 'start', 'stop', 'name', 'score', 'strand', 'tstart', 'tstop'])

In [3]:
genes = set(foo.name.apply(lambda x: x.split("_")[0]).values)

In [5]:
genes

{'ENSG00000106404.9',
 'ENSG00000107404.13',
 'ENSG00000129422.9',
 'ENSG00000132613.10',
 'ENSG00000213221.4',
 'ENSG00000225218.1',
 'ENSG00000240801.1'}

In [28]:
for gene in genes:
    cmd = "clipper  '-b' '/projects/ps-yeolab3/encode/analysis/encode_master/204_01_RBFOX2.merged.r2.bam'  '-s' 'hg19'  '-o' '/home/gpratt/clipper_peaks/204_01_RBFOX2.merged.r2.{}.peaks.bed'  '--save-pickle' --gene {} --debug".format(gene, gene)
    print gene,
    !$cmd
    
    !/home/gpratt/clipper_peaks/read_diff.sh $gene

ENSG00000132613.10 ENSG00000225218.1 ENSG00000129422.9 ENSG00000106404.9 ENSG00000107404.13 ENSG00000213221.4 ENSG00000240801.1


In [ ]:
            if genomic_start == 2245:
                print area_start, area_stop
                print interval.start + area_start, interval.start + area_stop
                print cur_pybedtools_interval
                for read in get_reads_in_interval_pysam(cur_pybedtools_interval, interval.start, read_locations):
                    if read.qname in interesting_reads:
                        print "really odd"

                        
3187 1670 3230 3230
3187 1670 3230 3230
3187 1670 3230 3230
3187 1670 3230 3230
3187 1670 3230 3230
3188 1670 3230 3230
3188 1670 3230 3230
3188 1670 3230 3230
3190 1670 3230 3230
3194 1670 3230 3230
3196 1670 3230 3230
3197 1670 3230 3230
3198 1670 3230 3230
3199 1670 3230 3230
3199 1670 3230 3230
3199 1670 3230 3230
3199 1670 3230 3230
3199 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3200 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230
3201 1670 3230 3230  
3201 1670 3230 3230


In [57]:
result = """FOX2_AFFSC_CLIP_S75

FOX2_AFFSC_IN_S76

FOX2_SS4_CLIP_S83

FOX2_SS4_IN_S84

FOX2_SS4_MN_CLIP_S91

FOX2_SS4_MN_IN_S92

FOX2_SS4in3B_CLIP_S87

FOX2_SS4in3B_CLIP_-AS45_S99

FOX2_SS4in3B_IN_S88

FOX2_SS4in3B_IN_AS46_S100

FOX2_SSIII_CLIP_S79

FOX2_SSIII_IN_S80

FOX2_TGIRT_CLIP_S95

FOX2_TGIRT_IN_S96

TDP_AFFSC_CLIP_S77

TDP_AFFSC_IN_S78

TDP_SS4_CLIP_S85

TDP_SS4_IN_S86

TDP_SS4_MN_CLIP_S93

TDP_SS4_MN_IN_S94

TDP_SS4in3B_CLIP_S89

TDP_SS4in3B_CLIP_AS47_S101

TDP_SS4in3B_IN_S90

TDP_SS4in3B_IN_AS48_S102

TDP_SSIII_CLIP_S81

TDP_SSIII_IN_S82

TDP_TGIRT_CLIP_S97

TDP_TGIRT_IN_S98""".split("\n")

In [59]:
result = [item for item in result if len(item) > 0]

In [61]:
",".join(result)

'FOX2_AFFSC_CLIP_S75,FOX2_AFFSC_IN_S76,FOX2_SS4_CLIP_S83,FOX2_SS4_IN_S84,FOX2_SS4_MN_CLIP_S91,FOX2_SS4_MN_IN_S92,FOX2_SS4in3B_CLIP_S87,FOX2_SS4in3B_CLIP_-AS45_S99,FOX2_SS4in3B_IN_S88,FOX2_SS4in3B_IN_AS46_S100,FOX2_SSIII_CLIP_S79,FOX2_SSIII_IN_S80,FOX2_TGIRT_CLIP_S95,FOX2_TGIRT_IN_S96,TDP_AFFSC_CLIP_S77,TDP_AFFSC_IN_S78,TDP_SS4_CLIP_S85,TDP_SS4_IN_S86,TDP_SS4_MN_CLIP_S93,TDP_SS4_MN_IN_S94,TDP_SS4in3B_CLIP_S89,TDP_SS4in3B_CLIP_AS47_S101,TDP_SS4in3B_IN_S90,TDP_SS4in3B_IN_AS48_S102,TDP_SSIII_CLIP_S81,TDP_SSIII_IN_S82,TDP_TGIRT_CLIP_S97,TDP_TGIRT_IN_S98'

In [63]:
result = """/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_CLIP_S75_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_CLIP_S75_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_IN_S76_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_IN_S76_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_CLIP_S83_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_CLIP_S83_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_CLIP_-AS45_S99_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_CLIP_-AS45_S99_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_CLIP_S87_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_CLIP_S87_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_IN_AS46_S100_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_IN_AS46_S100_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_IN_S88_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4in3B_IN_S88_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_IN_S84_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_IN_S84_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_MN_CLIP_S91_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_MN_CLIP_S91_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_MN_IN_S92_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_MN_IN_S92_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SSIII_CLIP_S79_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SSIII_CLIP_S79_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SSIII_IN_S80_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SSIII_IN_S80_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_TGIRT_CLIP_S95_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_TGIRT_CLIP_S95_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_TGIRT_IN_S96_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_TGIRT_IN_S96_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_AFFSC_CLIP_S77_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_AFFSC_CLIP_S77_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_AFFSC_IN_S78_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_AFFSC_IN_S78_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_CLIP_S85_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_CLIP_S85_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_CLIP_AS47_S101_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_CLIP_AS47_S101_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_CLIP_S89_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_CLIP_S89_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_IN_AS48_S102_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_IN_AS48_S102_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_IN_S90_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4in3B_IN_S90_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_IN_S86_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_IN_S86_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_MN_CLIP_S93_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_MN_CLIP_S93_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_MN_IN_S94_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SS4_MN_IN_S94_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SSIII_CLIP_S81_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SSIII_CLIP_S81_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SSIII_IN_S82_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_SSIII_IN_S82_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_TGIRT_CLIP_S97_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_TGIRT_CLIP_S97_L007_R2_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_TGIRT_IN_S98_L007_R1_001.fastq.gz
/projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/TDP_TGIRT_IN_S98_L007_R2_001.fastq.gz
""".split()

In [96]:
fn = result[0]

In [67]:
import os

In [94]:
fn.index?

In [103]:
def parse_fn(fn):
    fn = os.path.basename(fn)
    fn = fn.split("_")
    if "IN" in fn:
        max_index = fn.index("IN")
    elif "CLIP" in fn:
        max_index = fn.index("CLIP")
    else:
        print "neither"
    info = "_".join(fn[:max_index])
    read = fn[-2]
    return (info, read)
parse_fn(fn)

('FOX2_AFFSC', 'R1')

In [104]:
from itertools import groupby

In [105]:
foo = pd.Series(result)

In [111]:
for name, group in groupby(sorted(result, key=parse_fn), parse_fn):
    group = list(group)
    if len(group) > 2:
        for fn in group:
            print fn
    else:
        print "cat {} > /home/gpratt/projects/encode/data/encode_merged/{}_L000_{}_001.fastq.gz".format(" ".join(group), name[0], name[1])

cat /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_CLIP_S75_L007_R1_001.fastq.gz /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_IN_S76_L007_R1_001.fastq.gz > /home/gpratt/projects/encode/data/encode_merged/FOX2_AFFSC_L000_R1_001.fastq.gz
cat /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_CLIP_S75_L007_R2_001.fastq.gz /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_AFFSC_IN_S76_L007_R2_001.fastq.gz > /home/gpratt/projects/encode/data/encode_merged/FOX2_AFFSC_L000_R2_001.fastq.gz
cat /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_CLIP_S83_L007_R1_001.fastq.gz /projects/ps-yeolab/seqdata/igm-storage1.ucsd.edu/160927_K00180_0248_AHCT55BBXX/Data/Fastq/FOX2_SS4_IN_S84_L007_R1_001.fastq.gz > /home/gpratt/projects/encode/data/encode